# PDF Parsing example

This example shows how to use the PDF parsing skill to extract text from a PDF file.

First you have to start the broker and the pdf parsing skill. You can do this by running the following commands in the terminal:

```bash
make ENV=main build
pip install .

brokerio skills build pd3f
brokerio skills run pd3f --url "http://broker:4852" --network "nlp_api_main_default"
```

Then you can run the following code to extract text from a PDF file.

In [ ]:
# Running commands locally or install latest nlp-broker from Github
!pip install brokerio

In [1]:
# Imports and setup
from brokerio.client import Client, ClientTimeoutException
import os

broker_url = "http://127.0.0.1:4852"

In [2]:
# initialize client
client = Client(broker_url)

# start client (starts new thread and connect to the broker)
client.start()

# Let's clear the queue and look what skills are available
client.clear()
print("Skills: {}".format(client.skills))

Skills: [{'name': 'pd3f', 'nodes': 1}]


2024-03-28 15:47:59,927 SpawnProcess-3 Simple Client INFO     Attempting polling connection to http://127.0.0.1:4852/socket.io/?transport=polling&EIO=4
2024-03-28 15:47:59,941 SpawnProcess-3 Simple Client INFO     Polling connection accepted with {'sid': '_rhu_XjHRry5FfqbAAAY', 'upgrades': ['websocket'], 'pingTimeout': 20000, 'pingInterval': 25000}
2024-03-28 15:47:59,941 SpawnProcess-3 Simple Client INFO     Engine.IO connection established
2024-03-28 15:47:59,941 SpawnProcess-3 Simple Client INFO     Sending packet MESSAGE data 0{}
2024-03-28 15:47:59,941 SpawnProcess-3 Simple Client INFO     Attempting WebSocket upgrade to ws://127.0.0.1:4852/socket.io/?transport=websocket&EIO=4
2024-03-28 15:47:59,946 SpawnProcess-3 Simple Client INFO     WebSocket upgrade was successful
2024-03-28 15:47:59,948 SpawnProcess-3 Simple Client INFO     Received packet NOOP data 
2024-03-28 15:47:59,974 SpawnProcess-3 Simple Client INFO     Received packet MESSAGE data 2["skillUpdate",[{"name":"pd3f","n

In [3]:
# Get config for the pdf parsing skill
client.put({
    "event": "skillGetConfig",
    "data": {
        "name": "pd3f"
    }
})
results = client.wait_for_event("skillConfig", timeout=10)
if results:
    print(results['data'])
else:
    print("Timeout!")

{'name': 'pd3f', 'description': 'This is a pdf text extraction skill based on pd3f', 'input': {'data': {'pdf': {'type': 'string', 'description': 'Base64 encoded PDF file'}, 'params': {'type': 'object', 'description': 'Additional parameters for the pd3f extraction', 'required': False, 'properties': {'tables': {'type': 'boolean', 'description': 'extract tables via Parsr (with Camelot / Tabula), results into list of CSV strings'}, 'experimental': {'type': 'boolean', 'description': 'leave out duplicate text in headers / footers and turn footnotes to endnotes. Working unreliable right now'}, 'lang': {'type': 'string', 'description': 'set the language, de for German, en for English, es for Spanish, fr for French. Some fast (less accurate) models exists. So set multi-v0-fast to get fast model for German, French (and some other languages)'}, 'fast': {'type': 'boolean', 'description': 'Drop some Parsr steps to speed up computations'}}}}, 'example': {'pdf': 'base64 encoded pdf', 'params': {'tabl

2024-03-28 15:48:01,913 SpawnProcess-3 Simple Client INFO     Emitting event "skillGetConfig" [/]
2024-03-28 15:48:01,913 SpawnProcess-3 Simple Client INFO     Sending packet MESSAGE data 2["skillGetConfig",{"name":"pd3f"}]
2024-03-28 15:48:01,940 SpawnProcess-3 Simple Client INFO     Received packet MESSAGE data 2["skillConfig",{"name":"pd3f","description":"This is a pdf text extraction skill based on pd3f","input":{"data":{"pdf":{"type":"string","description":"Base64 encoded PDF file"},"params":{"type":"object","description":"Additional parameters for the pd3f extraction","required":false,"properties":{"tables":{"type":"boolean","description":"extract tables via Parsr (with Camelot / Tabula), results into list of CSV strings"},"experimental":{"type":"boolean","description":"leave out duplicate text in headers / footers and turn footnotes to endnotes. Working unreliable right now"},"lang":{"type":"string","description":"set the language, de for German, en for English, es for Spanish, 

In [6]:
# load and send pdf as base64
import base64
pdf_path = "2023.acl-demo.28v2.pdf"
with open(pdf_path, "rb") as f:
    pdf = base64.b64encode(f.read()).decode("utf-8")

skill = "pd3f"
event = 'skillRequest'
message_id = "test"
config = {
    "return_stats": True
}
data = {"pdf": pdf}
timeout = 10

try:
    result = client.request(skill, data, message_id, config=config, timeout=timeout)
    print(result)
except ClientTimeoutException as e:
    print(e)

2024-03-28 16:00:52,106 SpawnProcess-3 Simple Client INFO     Emitting event "skillRequest" [/]
2024-03-28 16:00:52,110 SpawnProcess-3 Simple Client INFO     Sending packet MESSAGE data 2["skillRequest",{"name":"pd3f","id":"test","config":{"return_stats":true},"data":{"pdf":"..."}}]
2024-03-28 16:00:53,400 SpawnProcess-3 Simple Client INFO     Received packet MESSAGE data 2["error",{"id":3317,"code":112,"error":{"code":112,"message":"Object of type LogRecord is not JSON serializable"}}]
2024-03-28 16:00:53,401 SpawnProcess-3 Simple Client INFO     Received event "error" [/]
2024-03-28 16:00:55,164 SpawnProcess-3 Simple Client INFO     Received packet PING data 
2024-03-28 16:00:55,164 SpawnProcess-3 Simple Client INFO     Sending packet PONG data 


Timeout while waiting for results


NameError: name 'ö' is not defined